In [1]:
from google.colab import drive
 
drive.mount('/gdrive')
# the project's folder
%cd '/gdrive/My Drive/callgraphCA'

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1z4fjsm8NcEBHt9yh6KPOaHwaKBYqpozg/callgraphCA


In [2]:
!ls -la "/gdrive/MyDrive/callgraphCA" #uzh -> zhaw

lrw------- 1 root root 0 Dec 14 09:16 /gdrive/MyDrive/callgraphCA -> /gdrive/.shortcut-targets-by-id/1z4fjsm8NcEBHt9yh6KPOaHwaKBYqpozg/callgraphCA


In [3]:
!pip install apyori
# https://github.com/ymoch/apyori
# https://medium.com/linkit-intecs/apriori-algorithm-in-data-mining-part-2-590d58e0998b


  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=a07d4835a334791133ee2fb1e4c31a1ccbe1ba51664168ac807070659cc5a45a
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [4]:
import pandas as pd
import sqlite3
from apyori import apriori
from re import search
from typing import List

# works with lists, not pandas, no nan values, apostrophe between values of transaction

# min_support -- The minimum support of relations (float).
# min_confidence -- The minimum confidence of relations (float).
# min_lift -- The minimum lift of relations (float).
# max_length -- The maximum length of the relation (integer).


In [5]:
from coupling_association_rules_utils import *

In [29]:
from imp import importlib
importlib.reload(coupling_association_rules_utils)
from coupling_association_rules_utils import show_transactions_containing_items

ImportError: ignored

In [6]:
#PATH_TO_ANALYTICS_DB = '/gdrive/My Drive/callgraphCA/JKQtPlotter_analytics.db'
#PATH_TO_ANALYTICS_DB = '/gdrive/My Drive/callgraphCA/glucosio-small.db'
PATH_TO_ANALYTICS_DB = '/gdrive/My Drive/callgraphCA/glucosio-android_analytics.db'

In [7]:
con_graph_db = sqlite3.connect(PATH_TO_ANALYTICS_DB)

In [8]:
cur = con_graph_db.cursor()

Test connection

In [ ]:
sql_statement = """SELECT * FROM git_commit LIMIT 2"""

In [ ]:
df = pd.read_sql_query(sql_statement, con_graph_db)

In [ ]:
print(df.head())

In [ ]:
sql_statement = """SELECT * FROM git_commit LIMIT 10"""

# Co-change matrix

## On commit and file level - distinct

In [9]:
sql_statement = """select 
--commit_hash,
GROUP_CONCAT(distinct("'" || file_name|| "'") )  as files_in_hash
from file_commit
group by commit_hash;"""
df = pd.read_sql_query(sql_statement, con_graph_db)
print(len(df))
# WORKS stack functionality removes NaNs to generate the nested list:
records_concats = pd.DataFrame(df['files_in_hash']).stack().groupby(level=0).apply(list).values.tolist()
records = []
for r in records_concats:
    records.append(list(r[0].split(sep=',')))

582


In [ ]:
print(df[0:5])

In [ ]:
records[0:12]

In [10]:
rules = apriori(records, min_confidence=0.1, min_support=0.1) # min_lift=0.2, 
rules_list = list(rules)
print(len(rules_list))

5


In [11]:
for r in rules_list:
  print(r)

RelationRecord(items=frozenset({"'DatabaseHandler.java'"}), support=0.15807560137457044, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'DatabaseHandler.java'"}), confidence=0.15807560137457044, lift=1.0)])
RelationRecord(items=frozenset({"'HelloActivity.java'"}), support=0.10309278350515463, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'HelloActivity.java'"}), confidence=0.10309278350515463, lift=1.0)])
RelationRecord(items=frozenset({"'MainActivity.java'"}), support=0.2869415807560137, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'MainActivity.java'"}), confidence=0.2869415807560137, lift=1.0)])
RelationRecord(items=frozenset({"'OverviewFragment.java'"}), support=0.15979381443298968, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'OverviewFragment.java'"}), confidence=0.15979381443298968, lift=1.0)])
RelationRecord(items=frozenset({"'Pref

In [12]:
rules = apriori(records, min_confidence=0.05, min_support=0.05) # min_lift=0.2, 
rules_list = list(rules)
print(len(rules_list))

15


In [13]:
for r in rules_list:
  print(r)

RelationRecord(items=frozenset({"'AddGlucoseActivity.java'"}), support=0.08075601374570447, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddGlucoseActivity.java'"}), confidence=0.08075601374570447, lift=1.0)])
RelationRecord(items=frozenset({"'AddGlucosePresenter.java'"}), support=0.061855670103092786, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddGlucosePresenter.java'"}), confidence=0.061855670103092786, lift=1.0)])
RelationRecord(items=frozenset({"'BackupActivity.java'"}), support=0.061855670103092786, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'BackupActivity.java'"}), confidence=0.061855670103092786, lift=1.0)])
RelationRecord(items=frozenset({"'DatabaseHandler.java'"}), support=0.15807560137457044, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'DatabaseHandler.java'"}), confidence=0.15807560137457044, lift=1.0)])
RelationReco

### prune records with 2 or more occurrences

In [14]:
print(len(records))
pruned_records = []
for r in records:
    if len(r)>2:
        pruned_records.append(r)

print(len(pruned_records))

582
218


In [19]:
pruned_records[0:10]

[["'HelloActivity.java'",
  "'LicenceActivity.java'",
  "'PreferencesActivity.java'",
  "'AssistantAdapter.java'",
  "'HistoryAdapter.java'",
  "'HomePagerAdapter.java'",
  "'HistoryFragment.java'",
  "'OverviewFragment.java'",
  "'AssistantPresenter.java'",
  "'HelloPresenter.java'",
  "'HistoryPresenter.java'",
  "'OverviewPresenter.java'",
  "'FormatDateTime.java'",
  "'ReadingTools.java'"],
 ["'DatabaseHandler.java'",
  "'OverviewFragment.java'",
  "'OverviewPresenter.java'"],
 ["'GlucosioApplication.java'",
  "'AddA1CActivity.java'",
  "'AddCholesterolActivity.java'",
  "'AddKetoneActivity.java'",
  "'AddPressureActivity.java'",
  "'AddWeightActivity.java'",
  "'OverviewFragment.java'",
  "'OverviewPresenter.java'",
  "'HelloActivityTest.java'"],
 ["'RemindersActivity.java'",
  "'RemindersAdapter.java'",
  "'DatabaseHandler.java'",
  "'Migration.java'",
  "'Reminder.java'"],
 ["'ExportActivity.java'", "'DatabaseHandler.java'", "'ExportPresenter.java'"],
 ["'AddGlucosePresenter.jav

In [15]:
rules = apriori(pruned_records, min_confidence=0.15, min_support=0.15) # min_lift=0.2, 
rules_list = list(rules)
print(len(rules_list))

8


In [16]:
for r in rules_list:
  print(r)

RelationRecord(items=frozenset({"'AddGlucoseActivity.java'"}), support=0.18807339449541285, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddGlucoseActivity.java'"}), confidence=0.18807339449541285, lift=1.0)])
RelationRecord(items=frozenset({"'DatabaseHandler.java'"}), support=0.2981651376146789, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'DatabaseHandler.java'"}), confidence=0.2981651376146789, lift=1.0)])
RelationRecord(items=frozenset({"'HistoryAdapter.java'"}), support=0.16055045871559634, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'HistoryAdapter.java'"}), confidence=0.16055045871559634, lift=1.0)])
RelationRecord(items=frozenset({"'HistoryFragment.java'"}), support=0.16055045871559634, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'HistoryFragment.java'"}), confidence=0.16055045871559634, lift=1.0)])
RelationRecord(items=froze

In [17]:
rules = apriori(pruned_records, min_confidence=0.1, min_support=0.1) # min_lift=0.2, 
rules_list = list(rules)
print(len(rules_list))

25


In [18]:
for r in rules_list:
  print(r)
  

RelationRecord(items=frozenset({"'AddCholesterolActivity.java'"}), support=0.10091743119266056, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddCholesterolActivity.java'"}), confidence=0.10091743119266056, lift=1.0)])
RelationRecord(items=frozenset({"'AddGlucoseActivity.java'"}), support=0.18807339449541285, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddGlucoseActivity.java'"}), confidence=0.18807339449541285, lift=1.0)])
RelationRecord(items=frozenset({"'AddGlucosePresenter.java'"}), support=0.13761467889908258, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddGlucosePresenter.java'"}), confidence=0.13761467889908258, lift=1.0)])
RelationRecord(items=frozenset({"'AddKetoneActivity.java'"}), support=0.11009174311926606, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({"'AddKetoneActivity.java'"}), confidence=0.11009174311926606, lift=1.0

In [ ]:
df.head(5)

### find transactions where this rules happened

In [14]:
print(df.loc[df['files_in_hash'] == "'PreferencesActivity.java','LocaleHelper.java'"])


                                      files_in_hash
123  'PreferencesActivity.java','LocaleHelper.java'
128  'PreferencesActivity.java','LocaleHelper.java'
144  'PreferencesActivity.java','LocaleHelper.java'


In [20]:
l_elem = ['OverviewFragment.java', 'DatabaseHandler.java']
show_transactions_containing_items(df, l_elem, print_elems=False)

Element count. Df len 582. 1: 93, 2: 25


In [21]:
l_elem = ['AddCholesterolActivity', 'AddA1CActivity']
show_transactions_containing_items(df, l_elem, print_elems=False)

Element count. Df len 582. 1: 22, 2: 14


In [22]:
l_elems = ["'AddKetoneActivity.java'", "'AddWeightActivity.java'", "'AddGlucoseActivity.java'", "'AddPressureActivity.java'"]
show_transactions_containing_items(df, l_elems, print_elems=False)

Element count. Df len 582. 1: 25, 2: 20, 3: 17, 4: 17


In [ ]:
for e in l_elems:
    print(e)

## On week

In [ ]:
# apyori.apriori needs apostrophes around each of the values of the transaction
sql_statement = """select
(strftime('%j', date(commit_commiter_datetime, '-3 days', 'weekday 4')) - 1) / 7 + 1 as iso_week,
GROUP_CONCAT("'" || file_name|| "'") as files_in_week
--GROUP_CONCAT(distinct("'" || file_name|| "'") ) as files_in_week
from file_commit
group by (strftime('%j', date(commit_commiter_datetime, '-3 days', 'weekday 4')) - 1) / 7 + 1;"""
df = pd.read_sql_query(sql_statement, con_graph_db)
print(len(df))
records = pd.DataFrame(df['files_in_hash']).stack().groupby(level=0).apply(list).values.tolist()
records = []
for r in records_concats:
    records.append(list(r[0].split(sep=',')))

In [ ]:
rules = apriori(records, min_confidence=0.1, min_support=0.1) # min_lift=0.2, 
rules_list = list(rules) # from generator save to list for further analysis
print(len(rules_list))

In [ ]:
rules = apriori(records, min_confidence=0.01, min_support=0.01) # min_lift=0.2, 
rules_list = list(rules) # from generator save to list for further analysis
print(len(rules_list))

In [ ]:
rules = apriori(records, min_confidence=0.001, min_support=0.001) # min_lift=0.2, 
rules_list = list(rules) # from generator save to list for further analysis
print(len(rules_list))

### slide time window

In [ ]:
rules = apriori(records, min_support=0.2, min_confidence=0.2) #, min_lift=2, min_length=2)
rules_list = list(rules)

In [ ]:
print(len(rules_list))

In [ ]:
for r in rules_list:
  print(r)

#Test stuff

In [ ]:
from apriori_python import apriori
itemSetList = [['eggs.java', 'bacon.java', 'soup.java'],
                ['eggs.java', 'bacon.java', 'apple.java'],
                ['soup.java', 'bacon.java', 'banana.java'],
                ['soup.java', '', 'bacon.java']
               ]
freqItemSet, rules = apriori(itemSetList, minSup=0.5, minConf=0.5)
print(rules)  
# [[{'beer'}, {'rice'}, 0.6666666666666666], [{'rice'}, {'beer'}, 1.0]]
# rules[0] --> rules[1], confidence = rules[2]